# 🎯 Sentiment Analysis - Liam Rosenior Chelsea Appointment

**Complete Pipeline: Data Preprocessing → EDA → Sentiment Labeling → ML Modeling**

---

## How to Run on Google Colab

1. **Open Google Colab**: https://colab.research.google.com

2. **Clone the repository**:
```python
!git clone https://github.com/albertabayor/analisis-sentiment-pelatih-baru-chelsea-liam-rosenior.git
%cd analisis-sentiment-pelatih-baru-chelsea-liam-rosenior
```

3. **Mount Google Drive** (optional - for saving outputs):
```python
from google.colab import drive
drive.mount('/content/drive')
```

4. **Install dependencies**:
```python
!pip install -q pandas numpy scikit-learn nltk vaderSentiment langdetect matplotlib seaborn wordcloud plotly
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
```

5. **Run this notebook** by pressing `Shift+Enter` on each cell

---

## Project Summary

- **Dataset**: 621 tweets about Liam Rosenior's appointment as Chelsea FC head coach
- **After Cleaning**: ~329 tweets (filtered @grok queries, non-English)
- **Sentiment Analysis**: VADER
- **ML Models**: Logistic Regression, Multinomial Naive Bayes
- **Best Model**: Logistic Regression (~57.58% accuracy)

## 1. Setup & Install Dependencies

In [ ]:
# Install required packages
!pip install -q pandas numpy scikit-learn nltk vaderSentiment langdetect matplotlib seaborn wordcloud plotly joblib

# Download NLTK data
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("[OK] Dependencies installed successfully!")

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect, LangDetectException
from wordcloud import WordCloud, STOPWORDS
import joblib

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Create output directories
os.makedirs('outputs/figures', exist_ok=True)
os.makedirs('outputs/tables', exist_ok=True)
os.makedirs('outputs/models', exist_ok=True)
os.makedirs('outputs/metrics', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

print("[OK] Libraries imported and directories created!")

## 3. Data Preprocessing Functions

In [ ]:
# ==================== PREPROCESSING FUNCTIONS ====================

def is_grok_query(tweet_content):
    """Check if a tweet is a question TO @grok"""
    if not isinstance(tweet_content, str):
        return False
    tweet_lower = tweet_content.lower().strip()
    if tweet_lower.startswith('@grok'):
        return True
    if '@grok' in tweet_lower:
        question_words = ['who is', 'what is', 'where', 'when', 'why', 'how', 'can you', 'tell me', 'explain']
        for qw in question_words:
            if qw in tweet_lower:
                return True
    return False

def is_english(text):
    """Detect if text is in English"""
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

def clean_text(text):
    """Clean tweet text"""
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#(\w+)', r'\1', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_data(df):
    """Complete preprocessing pipeline"""
    print(f"Original tweets: {len(df)}")
    
    # Remove empty tweets
    df = df.dropna(subset=['Tweet Content'])
    df = df[df['Tweet Content'].str.strip() != '']
    print(f"After removing empty: {len(df)}")
    
    # Remove duplicates
    df = df.drop_duplicates(subset=['Tweet Content'], keep='first')
    print(f"After removing duplicates: {len(df)}")
    
    # Filter @grok queries
    is_grok_mask = df['Tweet Content'].apply(is_grok_query)
    df = df[~is_grok_mask]
    print(f"After removing @grok queries: {len(df)}")
    
    # Filter English only
    is_english_mask = df['Tweet Content'].apply(is_english)
    df = df[is_english_mask]
    print(f"After filtering English: {len(df)}")
    
    # Add engagement features
    numeric_cols = ['Views', 'Likes', 'Retweets', 'Replies']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    df['engagement_score'] = df['Likes'] + (df['Retweets'] * 2) + (df['Replies'] * 3)
    df['tweet_length'] = df['Tweet Content'].apply(lambda x: len(str(x)))
    df['word_count'] = df['Tweet Content'].apply(lambda x: len(str(x).split()))
    
    # Clean text
    df['Tweet Content Cleaned'] = df['Tweet Content'].apply(clean_text)
    
    print(f"\n[OK] Preprocessing complete! Final: {len(df)} tweets")
    return df

print("[OK] Preprocessing functions defined!")

## 4. Load & Preprocess Data

In [ ]:
# Load raw data
if os.path.exists('data/raw/tweets.csv'):
    df = pd.read_csv('data/raw/tweets.csv')
    print(f"Loaded {len(df)} tweets from data/raw/tweets.csv")
else:
    # Create sample data if file doesn't exist
    print("Creating sample dataset...")
    sample_data = {
        'Tweet Content': [
            "Great appointment for Chelsea! Liam Rosenior is the right choice!",
            "I'm skeptical about this appointment. Has he proven himself?",
            "Neutral opinion. Let's wait and see how it goes.",
            "@grok who is Liam Rosenior?",
            "Chelsea making another questionable decision."
        ] * 100  # Multiply for demo
    }
    df = pd.DataFrame(sample_data)
    # Add random engagement metrics
    np.random.seed(42)
    df['Views'] = np.random.randint(100, 10000, len(df))
    df['Likes'] = np.random.randint(0, 500, len(df))
    df['Retweets'] = np.random.randint(0, 100, len(df))
    df['Replies'] = np.random.randint(0, 50, len(df))
    print(f"Created sample dataset: {len(df)} tweets")

# Preprocess data
df = preprocess_data(df)

# Save cleaned data
df.to_csv('data/processed/tweets_cleaned.csv', index=False)
print("Saved to data/processed/tweets_cleaned.csv")

## 5. Sentiment Analysis (VADER)

In [ ]:
# Apply VADER sentiment analysis
analyzer = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
    if not isinstance(text, str):
        text = str(text)
    return analyzer.polarity_scores(text)

print("Applying VADER sentiment analysis...")
vader_scores = df['Tweet Content'].apply(get_vader_sentiment)

df['vader_neg'] = vader_scores.apply(lambda x: x['neg'])
df['vader_neu'] = vader_scores.apply(lambda x: x['neu'])
df['vader_pos'] = vader_scores.apply(lambda x: x['pos'])
df['vader_compound'] = vader_scores.apply(lambda x: x['compound'])

# Categorize sentiment
def categorize_sentiment(compound):
    if compound >= 0.05:
        return 'Positive'
    elif compound <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['vader_compound'].apply(categorize_sentiment)

# Save labeled data
df.to_csv('data/processed/tweets_labeled.csv', index=False)

# Print distribution
print("\n" + "="*50)
print("SENTIMENT DISTRIBUTION")
print("="*50)
sentiment_counts = df['sentiment'].value_counts()
for sent, count in sentiment_counts.items():
    pct = count/len(df)*100
    print(f"{sent}: {count} ({pct:.1f}%)")
print("="*50)

# Plot distribution
fig, ax = plt.subplots(figsize=(8, 6))
colors = ['#2ecc71', '#95a5a6', '#ff6b6b']
sentiment_counts.plot(kind='pie', autopct='%1.1f%%', colors=colors, ax=ax, startangle=90)
ax.set_title('Sentiment Distribution - Liam Rosenior Appointment', fontsize=14, fontweight='bold')
ax.set_ylabel('')
plt.tight_layout()
plt.savefig('outputs/figures/sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("[OK] Sentiment analysis complete!")

## 6. Exploratory Data Analysis

In [ ]:
# WordCloud
print("Generating WordCloud...")
all_text = ' '.join(df['Tweet Content'].astype(str).tolist())
wordcloud = WordCloud(width=1200, height=600, background_color='white',
                      stopwords=STOPWORDS, max_words=100, colormap='viridis').generate(all_text)

fig, ax = plt.subplots(figsize=(15, 8))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis('off')
ax.set_title('WordCloud - Most Frequent Words', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('outputs/figures/wordcloud.png', dpi=300, bbox_inches='tight')
plt.show()

# Engagement by Sentiment
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='sentiment', y='engagement_score', data=df,
            order=['Negative', 'Neutral', 'Positive'], palette='Set2', ax=ax)
ax.set_title('Engagement Score by Sentiment', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('outputs/figures/engagement_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

print("[OK] EDA complete!")

## 7. Machine Learning Modeling

In [ ]:
# Prepare features and labels
X = df['Tweet Content Cleaned']
y = df['sentiment']
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
y_encoded = y.map(label_map)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set: {len(X_train)} tweets")
print(f"Test set: {len(X_test)} tweets")

# TF-IDF Vectorization
print("\nCreating TF-IDF features...")
tfidf = TfidfVectorizer(max_features=500, ngram_range=(1, 2), stop_words='english', min_df=2, max_df=0.95)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print(f"TF-IDF features: {X_train_tfidf.shape[1]}")

# Count Vectorization
print("\nCreating Count features...")
count_vec = CountVectorizer(max_features=500, stop_words='english', min_df=2, max_df=0.95)
X_train_count = count_vec.fit_transform(X_train)
X_test_count = count_vec.transform(X_test)
print(f"Count features: {X_train_count.shape[1]}")

In [ ]:
# Train Logistic Regression
print("\n" + "="*50)
print("TRAINING LOGISTIC REGRESSION")
print("="*50)

lr = LogisticRegression(solver='lbfgs', max_iter=200, C=1.0, random_state=42)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_test_tfidf)

lr_accuracy = accuracy_score(y_test, y_pred_lr)
lr_precision = precision_score(y_test, y_pred_lr, average='weighted', zero_division=0)
lr_recall = recall_score(y_test, y_pred_lr, average='weighted', zero_division=0)
lr_f1 = f1_score(y_test, y_pred_lr, average='weighted', zero_division=0)

print(f"Accuracy: {lr_accuracy:.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")

# Train Naive Bayes
print("\n" + "="*50)
print("TRAINING MULTINOMIAL NAIVE BAYES")
print("="*50)

nb = MultinomialNB(alpha=1.0, fit_prior=True)
nb.fit(X_train_count, y_train)
y_pred_nb = nb.predict(X_test_count)

nb_accuracy = accuracy_score(y_test, y_pred_nb)
nb_precision = precision_score(y_test, y_pred_nb, average='weighted', zero_division=0)
nb_recall = recall_score(y_test, y_pred_nb, average='weighted', zero_division=0)
nb_f1 = f1_score(y_test, y_pred_nb, average='weighted', zero_division=0)

print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1-Score: {nb_f1:.4f}")

In [ ]:
# Model Comparison
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)

comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Multinomial Naive Bayes'],
    'Accuracy': [lr_accuracy, nb_accuracy],
    'Precision': [lr_precision, nb_precision],
    'Recall': [lr_recall, nb_recall],
    'F1-Score': [lr_f1, nb_f1]
})

print(comparison.to_string(index=False))

# Best model
best_idx = comparison['Accuracy'].idxmax()
best_model = comparison.loc[best_idx, 'Model']
print(f"\n[OK] Best Model: {best_model} (Accuracy: {comparison.loc[best_idx, 'Accuracy']:.4f})")

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(2)
width = 0.2
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
for i, metric in enumerate(metrics):
    ax.bar(x + i*width, comparison[metric], width, label=metric)
ax.set_xticks(x + width*1.5)
ax.set_xticklabels(comparison['Model'], rotation=15)
ax.set_ylim([0, 1])
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('outputs/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Save comparison
comparison.to_csv('outputs/tables/model_comparison.csv', index=False)
print("\n[OK] Model comparison saved to outputs/tables/model_comparison.csv")

In [ ]:
# Confusion Matrix for best model
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LR Confusion Matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Logistic Regression Confusion Matrix')

# NB Confusion Matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_title('Naive Bayes Confusion Matrix')

plt.tight_layout()
plt.savefig('outputs/figures/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("[OK] Confusion matrices saved!")

## 8. Save Models

In [ ]:
# Save trained models
joblib.dump(lr, 'outputs/models/logistic_regression.pkl')
joblib.dump(nb, 'outputs/models/naive_bayes.pkl')
joblib.dump(tfidf, 'outputs/models/tfidf_vectorizer.pkl')
joblib.dump(count_vec, 'outputs/models/count_vectorizer.pkl')

print("[OK] Models saved to outputs/models/")

# Save final summary
summary = {
    'total_tweets': len(df),
    'positive_tweets': (df['sentiment']=='Positive').sum(),
    'neutral_tweets': (df['sentiment']=='Neutral').sum(),
    'negative_tweets': (df['sentiment']=='Negative').sum(),
    'lr_accuracy': lr_accuracy,
    'nb_accuracy': nb_accuracy,
    'best_model': best_model
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv('outputs/tables/final_summary.csv', index=False)

print("[OK] Final summary saved to outputs/tables/final_summary.csv")

# Print final summary
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"Total Tweets Analyzed: {len(df)}")
print(f"  - Positive: {summary['positive_tweets']} ({summary['positive_tweets']/len(df)*100:.1f}%)")
print(f"  - Neutral: {summary['neutral_tweets']} ({summary['neutral_tweets']/len(df)*100:.1f}%)")
print(f"  - Negative: {summary['negative_tweets']} ({summary['negative_tweets']/len(df)*100:.1f}%)")
print(f"\nBest Model: {best_model}")
print(f"Logistic Regression Accuracy: {lr_accuracy:.4f}")
print(f"Naive Bayes Accuracy: {nb_accuracy:.4f}")
print("="*60)

print("\n🎉 ANALYSIS COMPLETE!")
print("\nOutput files saved to:")
- outputs/figures/ (visualizations)
- outputs/tables/ (CSV tables)
- outputs/models/ (trained models)")

## 📁 Output Files

All outputs are saved in the `outputs/` folder:

**Figures:**
- `sentiment_distribution.png` - Pie chart of sentiment
- `wordcloud.png` - Word frequency visualization
- `engagement_by_sentiment.png` - Engagement analysis
- `model_comparison.png` - Model performance comparison
- `confusion_matrices.png` - Confusion matrices for both models

**Tables:**
- `model_comparison.csv` - Model performance metrics
- `final_summary.csv` - Overall analysis summary

**Models:**
- `logistic_regression.pkl` - Trained LR model
- `naive_bayes.pkl` - Trained NB model
- `tfidf_vectorizer.pkl` - TF-IDF vectorizer
- `count_vectorizer.pkl` - Count vectorizer